In [ ]:
import numpy as np

First order eikonal update function

In [ ]:
def update_eikonal_2D(i, j, u, F, h, N):
    # LEFTMOST OR RIGHTMOST BORDERS
    if i == 0:
        a = u[i + 1, j]
    elif i == N - 1:
        a = u[i - 1, j]
    else:
        a = min(u[i + 1, j], u[i - 1, j])

    # UPPERMOST OR LOWERMOST BORDERS
    if j == 0:
        b = u[i, j + 1]
    elif j == N - 1:
        b = u[i, j - 1]
    else:
        b = min(u[i, j + 1], u[i, j - 1])

    # EIKONAL UPDATE
    if abs(a - b) < h * F[i, j]:
        u_new = 0.5 * (a + b + np.sqrt((2 * (h * F[i, j]) ** 2) - (a - b) ** 2))
    else:
        u_new = h * F[i, j] + min(a, b)

    return min(u[i, j], u_new)

Second order eikonal update functions

In [ ]:
def update_coefficients(switch, alpha, alpha_prime, t, a, b, c):
  alphatrue   = (switch * alpha + (1 - switch) * alpha_prime)
  beta        = -(2 * t * alphatrue)
  gamma       =  (t**2) * alphatrue
  a           += alphatrue
  b           += beta
  c           += gamma

  return a, b, c

def update_eikonal_second_order(i, j, u, F, isSet, h, N):

  a = 0
  b = 0
  c = -(F[i, j]**2)

  alpha = 1 / (h**2)
  alpha_prime = 9 / (4 * (h**2))

  switch = True
  # --- Leftmost boundary
  if i == 0:
    ta          = u[i + 1, j]
    a, b, c     = update_coefficients(switch, alpha, alpha_prime, ta, a, b, c)
  # --- Leftmost but one boundary
  elif i == 1:
    ta = min(u[i + 1, j], u[i - 1, j])
    a, b, c     = update_coefficients(switch, alpha, alpha_prime, ta, a, b, c)
  # --- Rightmost boundary
  elif i == N - 1:
    ta = u[i - 1, j]
    a, b, c     = update_coefficients(switch, alpha, alpha_prime, ta, a, b, c)
  # --- Rightmost but one boundary
  elif i == N - 2:
    ta = min(u[i + 1, j], u[i - 1, j])
    a, b, c     = update_coefficients(switch, alpha, alpha_prime, ta, a, b, c)
  else:
    switch1 = False
    switch2 = False
    if isSet[i - 2, j] & isSet[i - 1, j] & (u[i - 2, j] < u[i - 1, j]):
      tprimea1 = (4 * u[i - 1, j] - u[i - 2, j]) / 3
    else:
      tprimea1 = u[i - 1, j]
      switch1  = True
    if isSet[i + 2, j] & isSet[i + 1, j] & (u[i + 2, j] < u[i + 1, j]):
      tprimea2 = (4 * u[i + 1, j] - u[i + 2, j]) / 3
    else:
      tprimea2 = u[i + 1, j]
      switch2  = True
    if tprimea1 < tprimea2:
      ta = tprimea1
      switch = switch1
    else:
      ta = tprimea2
      switch = switch2
    a, b, c     = update_coefficients(switch, alpha, alpha_prime, ta, a, b, c)

  switch = True
  # --- Lowermost boundary
  if j == 0:
    tb          = u[i, j + 1]
    a, b, c     = update_coefficients(switch, alpha, alpha_prime, tb, a, b, c)
  # --- Lowermost but one boundary
  elif j == 1:
    tb = min(u[i, j + 1], u[i, j - 1])
    a, b, c     = update_coefficients(switch, alpha, alpha_prime, tb, a, b, c)
  # --- Rightmost boundary
  elif j == N - 1:
    tb = u[i, j - 1]
    a, b, c     = update_coefficients(switch, alpha, alpha_prime, tb, a, b, c)
  # --- Rightmost but one boundary
  elif j == N - 2:
    tb = min(u[i, j + 1], u[i, j - 1])
    a, b, c     = update_coefficients(switch, alpha, alpha_prime, tb, a, b, c)
  else:
    switch1 = False
    switch2 = False
    if isSet[i, j - 2] & isSet[i, j - 1] & (u[i, j - 2] < u[i, j - 1]):
      tprimea1 = (4 * u[i, j - 1] - u[i, j - 2]) / 3
    else:
      tprimea1 = u[i, j - 1]
      switch1  = True
    if isSet[i, j + 2] & isSet[i, j + 1] & (u[i, j + 2] < u[i, j + 1]):
      tprimea2 = (4 * u[i, j + 1] - u[i, j + 2]) / 3
    else:
      tprimea2 = u[i, j + 1]
      switch2  = True
    if tprimea1 < tprimea2:
      tb = tprimea1
      switch = switch1
    else:
      tb = tprimea2
      switch = switch2
    a, b, c     = update_coefficients(switch, alpha, alpha_prime, tb, a, b, c)

  # EIKONAL UPDATE
  discriminant = b**2 - 4 * a * c
  if discriminant >= 0:
    u_new = (-b + np.sqrt(discriminant)) / (2 * a)
    result = min(u[i, j], u_new)
  else:
    result = h * F[i, j] + min(ta, tb)

  return result